In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras import layers
from keras import models
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from os import listdir, makedirs
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers, regularizers

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
RESOLUTION = 150
BATCH_SIZE=64

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        "../input/state-farm-distracted-driver-detection/train/",
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 22424 images belonging to 10 classes.


In [4]:
path = "../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
conv_base = VGG16(include_top=False, input_shape=(150, 150, 3), weights=path) 

## Feature Extraction

In [5]:
def vgg16_pretrained_model(model_vgg, dropout_=False, regularizer_=False, regularizer_weight=0.001):
  
    model = Sequential()
    model.add(model_vgg)
    model.add(Flatten())
    if dropout_:
        model.add(Dropout(0.5))
    if regularizer_:
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=regularizer_weight,
                                                                                            l2=regularizer_weight)))
    else:
        model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

    return model

In [6]:
model = vgg16_pretrained_model(conv_base, dropout_=False, regularizer_=False, regularizer_weight=0.001)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 16,814,666
Trainable params: 16,814,666
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=50)

## Fine Tunning

In [ ]:
def build_vgg16_pretrained_fine_tunning(model_vgg, first_trainable_layer='block5_conv3',
                                       with_dropout=False,
                                       with_regularizer=False,
                                       regularizer_weight=0.001):
    set_trainable = False
    for layer in model_vgg.layers:
        if layer.name == first_trainable_layer:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    model = Sequential()
    model.add(model_vgg)
    model.add(Flatten())
    if with_dropout:
        model.add(Dropout(0.5))
    if with_regularizer:
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=regularizer_weight,
                                                                                            l2=regularizer_weight)))
    else:
        model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax')) # for multiclass single-label classification
    model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc']) # low learn. rate
    
    return model